In [ ]:
!pip install evaluate bert_score

In [1]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
from transformers import BlipProcessor, BlipForQuestionAnswering, BartTokenizer, BartForConditionalGeneration
from PIL import Image
import torch
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
# import evaluate
# from bert_score import score as bert_score

from transformers import (
    BlipForQuestionAnswering,
    BlipProcessor
)

model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

2025-05-13 10:02:18.679234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747130538.857470      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747130538.911222      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [5]:
df = pd.read_csv('/kaggle/input/combined-vqa/flattened_vqa_with_metadata.csv')
df = df.sample(frac=0.4, random_state=42).copy()

In [ ]:
# import os
# import io
# import torch
# import warnings
# from PIL import Image
# import contextlib
# import pandas as pd


# # Assuming processor and model are already initialized and on CUDA
# # image_base_path should be the root of your image folder
# image_base_path = "/kaggle/input/abo-small/small"

# skipped = 0
# skip_logs = []

# for idx, row in df.iterrows():
#     question = row["question"]
#     gt_answer = str(row["answer"]).strip().lower()
#     image_rel_path = row["image_path"]

#     full_path = image_rel_path  # Already contains full relative path
#     if not os.path.exists(full_path):
#         skip_logs.append(f"[SKIP] Image not found: {full_path}")
#         skipped += 1
#         continue

#     try:
#         image = Image.open(full_path).convert("RGB")
#     except Exception as e:
#         skip_logs.append(f"[SKIP] Failed to load image: {full_path} | Error: {e}")
#         skipped += 1
#         continue

#     with warnings.catch_warnings():
#         warnings.simplefilter("ignore")
#         with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
#             inputs = processor(images=image, text=question, return_tensors="pt")
#             output = model.generate(**inputs, max_new_tokens=10)
#             pred_answer = processor.decode(output[0], skip_special_tokens=True).strip().lower()

#     print(f"Q: {question}\nPredicted: {pred_answer} | Ground Truth: {gt_answer}\n")


In [ ]:
!pip install -U bitsandbytes

In [ ]:
!pip install peft

In [7]:
# import torch
# from peft import get_peft_model, LoraConfig, TaskType




# # LoRA configuration
# lora_config = LoraConfig(
#     r=4,
#     lora_alpha=16,
#     target_modules=["q", "v"],  # 'q' and 'v' are common; confirm exact target modules if needed
#     lora_dropout=0.1,
#     bias="none",
#     task_type=TaskType.SEQ_2_SEQ_LM
# )

# # Apply LoRA
# model = get_peft_model(model, lora_config)

# # Print number of trainable parameters
# def print_trainable_params(model):
#     trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
#     total = sum(p.numel() for p in model.parameters())
#     print(f"Trainable params: {trainable:,} / {total:,} ({100 * trainable / total:.2f}%)")

# print_trainable_params(model)

from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training

lora_config = LoraConfig(
    # use_rslora = True,
    init_lora_weights = "pissa",
    r=16,
    lora_alpha=16,
    target_modules=["query", "value", "projection", "fc1", "fc2", "key", "qkv"],
    lora_dropout=0.1,
    bias="none",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 5,898,240 || all params: 390,570,812 || trainable%: 1.5102


In [8]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import requests
from transformers import BlipProcessor, BlipForQuestionAnswering
import torch
from PIL import Image
from torch.utils.data import DataLoader
from tqdm import tqdm
import pickle


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Dataset class modification for CSV format
class CSVVQADataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, processor):
        self.data = dataframe
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        image = Image.open(item['image_path']).convert("RGB")
        text = item['question']
        answer = item['answer']
        if isinstance(answer, list):
            answer = " ".join(map(str, answer))  # convert list items to str if needed
        elif not isinstance(answer, str):
            answer = str(answer)
        
        if isinstance(text, list):
            text = " ".join(map(str, text))  # convert list items to str if needed
        elif not isinstance(text, str):
            text = str(text)
            
        encoding = self.processor(
            image, 
            text, 
            max_length = 128,
            padding="max_length", 
            truncation=True, 
            return_tensors="pt"
        )
        
        labels = self.processor.tokenizer(
            answer, 
            max_length= 128, 
            padding="max_length", 
            return_tensors='pt'
        )
        
        encoding["labels"] = labels["input_ids"].squeeze()
        for k,v in encoding.items():  
            encoding[k] = v.squeeze()
        return encoding

# Load and split data
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)

# Create datasets
train_dataset = CSVVQADataset(train_df, processor)
valid_dataset = CSVVQADataset(valid_df, processor)

# Create dataloaders
batch_size = 3
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True,  # Enable shuffling
    pin_memory=True
)
valid_dataloader = DataLoader(
    valid_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    pin_memory=True
)


optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scaler = torch.cuda.amp.GradScaler()

# Training Loop
min_eval_loss = float("inf")
tracking_information = []
patience = 3
early_stopping_hook = 0

for epoch in range(2):
    model.train()
    train_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1} Training"):
        for k in batch:
            batch[k] = batch[k].to(device)
        with torch.cuda.amp.autocast():
            outputs = model(**batch)
            loss = outputs.loss
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item()
    
    model.eval()
    eval_loss = 0
    with torch.no_grad():
        for batch in tqdm(valid_dataloader, desc=f"Epoch {epoch+1} Validation"):
            for k in batch:
                batch[k] = batch[k].to(device)
            with torch.cuda.amp.autocast():
                outputs = model(**batch)
                loss = outputs.loss
            eval_loss += loss.item()
    
    train_loss /= len(train_dataloader)
    eval_loss /= len(valid_dataloader)
    tracking_information.append((train_loss, eval_loss))
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Eval Loss = {eval_loss:.4f}")

    if eval_loss < min_eval_loss:
        min_eval_loss = eval_loss
        model.save_pretrained("Model/blip-lora-saved")
        model.save_pretrained("/kaggle/working/blip-lora-saved")
        print("✅ Model saved.")
        early_stopping_hook = 0
    else:
        early_stopping_hook += 1
        if early_stopping_hook >= patience:
            print("⏹️ Early stopping.")
            break

pickle.dump(tracking_information, open("tracking_info.pkl", "wb"))
print("🎉 Finetuning complete!")


/tmp/ipykernel_31/266748961.py:85: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1 Training:   0%|          | 0/16979 [00:00<?, ?it/s]/tmp/ipykernel_31/266748961.py:99: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1 Validation:   0%|          | 0/4245 [00:00<?, ?it/s]/tmp/ipykernel_31/266748961.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1 Validation: 100%|██████████| 4245/4245 [12:33<00:00,  5.64it/s]


Epoch 1: Train Loss = 8.5978, Eval Loss = 8.5326
✅ Model saved.


Epoch 2 Validation: 100%|██████████| 4245/4245 [12:28<00:00,  5.67it/s]


Epoch 2: Train Loss = 8.5295, Eval Loss = 8.5281
✅ Model saved.
🎉 Finetuning complete!


In [ ]:
from transformers import BlipProcessor, BlipForQuestionAnswering
from peft import PeftModel, PeftConfig
import torch

# Load processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

# Load base model
base_model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, "Model/blip-lora-saved").to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

In [ ]:
# Load your test CSV (replace with actual path if needed)
test_df = pd.read_csv("/kaggle/input/combined-vqa/flattened_vqa_with_metadata.csv")  # or whatever test set you want
test_df = test_df.sample(n=100, random_state=42).copy()  # 🔁 Adjust sample size if needed

# Prepare predictions
predictions = []

for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Running Inference"):
    try:
        image_path = row['image_path']
        question = str(row['question'])

        image = Image.open(image_path).convert("RGB")

        # Preprocess
        inputs = processor(image, question, return_tensors="pt").to(device)

        # Generate answer
        with torch.no_grad():
            output = model.generate(**inputs)

        answer = processor.decode(output[0], skip_special_tokens=True)
        predictions.append(answer)
    except Exception as e:
        print(f"❌ Error with {image_path}: {e}")
        predictions.append("")

# Save predictions to CSV
test_df['predicted_answer'] = predictions
test_df.to_csv("blip_finetuned_predictions.csv", index=False)
print("✅ Predictions saved to blip_finetuned_predictions.csv")


In [ ]:
import os
import io
import torch
import warnings
from PIL import Image
import contextlib
import pandas as pd


# Limit to first 5 rows
df = df.head(5)

# Assuming processor and model are already initialized and on CUDA
# image_base_path should be the root of your image folder
image_base_path = "/kaggle/input/abo-small/small"

skipped = 0
skip_logs = []

for idx, row in df.iterrows():
    question = row["question"]
    gt_answer = str(row["answer"]).strip().lower()
    image_rel_path = row["image_path"]

    full_path = image_rel_path  # Already contains full relative path
    if not os.path.exists(full_path):
        skip_logs.append(f"[SKIP] Image not found: {full_path}")
        skipped += 1
        continue

    try:
        image = Image.open(full_path).convert("RGB")
    except Exception as e:
        skip_logs.append(f"[SKIP] Failed to load image: {full_path} | Error: {e}")
        skipped += 1
        continue

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
            inputs = processor(images=image, text=question, return_tensors="pt")
            output = model.generate(**inputs, max_new_tokens=10)
            pred_answer = processor.decode(output[0], skip_special_tokens=True).strip().lower()

    print(f"Q: {question}\nPredicted: {pred_answer} | Ground Truth: {gt_answer}\n")


In [1]:
from transformers import BlipProcessor, BlipForQuestionAnswering
from peft import PeftModel, PeftConfig
import torch

# Load processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

# Load base model
base_model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, "/kaggle/input/16-16-full-no-dense/transformers/default/1").to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

2025-05-13 19:08:29.808842: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747163310.035460      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747163310.099833      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

PeftModel(
  (base_model): LoraModel(
    (model): BlipForQuestionAnswering(
      (vision_model): BlipVisionModel(
        (embeddings): BlipVisionEmbeddings(
          (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        )
        (encoder): BlipEncoder(
          (layers): ModuleList(
            (0-11): 12 x BlipEncoderLayer(
              (self_attn): BlipAttention(
                (dropout): Dropout(p=0.0, inplace=False)
                (qkv): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=2304, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_features=2304, bias=False)
                  )
   

In [2]:
from PIL import Image
import pandas as pd
import torch
from PIL import Image
from tqdm import tqdm
from transformers import BlipProcessor, BlipForQuestionAnswering
from peft import PeftModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load your test CSV (replace with actual path if needed)
test_df = pd.read_csv("/kaggle/input/combined-vqa/flattened_vqa_with_metadata.csv")  # or whatever test set you want
test_df = test_df.sample(n=30000, random_state=42).copy()  # 🔁 Adjust sample size if needed

# Prepare predictions
predictions = []

for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Running Inference"):
    try:
        image_path = row['image_path']
        question = str(row['question'])

        image = Image.open(image_path).convert("RGB")

        # Preprocess
        inputs = processor(image, question, return_tensors="pt").to(device)

        # Generate answer
        with torch.no_grad():
            output = model.generate(**inputs)

        answer = processor.decode(output[0], skip_special_tokens=True)
        predictions.append(answer)
    except Exception as e:
        print(f"❌ Error with {image_path}: {e}")
        predictions.append("")

# Save predictions to CSV
test_df['predicted_answer'] = predictions
test_df.to_csv("blip_16-16-full-no-dense-epoch2-finetuned_predictions.csv", index=False)
print("✅ Predictions saved to blip_8-16-qkv-finetuned_predictions.csv")


Running Inference: 100%|██████████| 30000/30000 [1:01:10<00:00,  8.17it/s]


✅ Predictions saved to blip_8-16-qkv-finetuned_predictions.csv


In [3]:
!pip install bert-score rouge-score nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.2 MB/s eta 0:00:00:00:0100:01
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=57fd8c396358b2b40fa16c73bf1a5e10d48fe5850ff10d46104ed17ea8862e50
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1

In [4]:
import pandas as pd
from sklearn.metrics import f1_score
from bert_score import score as bert_score
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
# from bart_score import BARTScorer
from tqdm import tqdm

tqdm.pandas()


In [5]:
# Load CSV
df = pd.read_csv("/kaggle/working/blip_16-16-full-no-dense-epoch2-finetuned_predictions.csv")  # Replace with your CSV file path

# Clean text (optional but recommended)
def normalize_text(s):
    return str(s).strip().lower()

df['answer'] = df['answer'].astype(str).apply(normalize_text)
df['predicted_answer'] = df['predicted_answer'].astype(str).apply(normalize_text)

# Accuracy
accuracy = (df['answer'] == df['predicted_answer']).mean()

# BERTScore (Precision/Recall/F1)
P, R, F1 = bert_score(df['predicted_answer'].tolist(), df['answer'].tolist(), lang='en', verbose=True)
bert_f1 = F1.mean().item()

# ROUGE Score
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
df['rougeL'] = df.progress_apply(lambda row: rouge.score(row['answer'], row['predicted_answer'])['rougeL'].fmeasure, axis=1)
avg_rougeL = df['rougeL'].mean()

# F1 Score (Token-level macro F1)
def compute_token_f1(pred, gt):
    pred_tokens = pred.split()
    gt_tokens = gt.split()
    common = set(pred_tokens) & set(gt_tokens)
    if not pred_tokens or not gt_tokens:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(gt_tokens)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

df['token_f1'] = df.progress_apply(lambda row: compute_token_f1(row['predicted_answer'], row['answer']), axis=1)
avg_token_f1 = df['token_f1'].mean()

# BLEU Score
smooth_fn = SmoothingFunction().method1
df['bleu'] = df.progress_apply(lambda row: sentence_bleu([row['answer'].split()], row['predicted_answer'].split(), smoothing_function=smooth_fn), axis=1)
avg_bleu = df['bleu'].mean()

# # BARTScore (optional; needs large model and GPU)
# bart_scorer = BARTScorer(device='cuda' if torch.cuda.is_available() else 'cpu', checkpoint='facebook/bart-large-cnn')
# df['bartscore'] = df.progress_apply(lambda row: bart_scorer.score([row['answer']], [row['ground_truth']])[0], axis=1)
# avg_bartscore = df['bartscore'].mean()

print(f"Accuracy      : {accuracy:.4f}")
print(f"BERTScore F1  : {bert_f1:.4f}")
print(f"ROUGE-L       : {avg_rougeL:.4f}")
print(f"Token F1      : {avg_token_f1:.4f}")
print(f"BLEU          : {avg_bleu:.4f}")
# print(f"BARTScore     : {avg_bartscore:.4f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

computing greedy matching.


  0%|          | 0/469 [00:00<?, ?it/s]

done in 7.43 seconds, 4036.15 sentences/sec


100%|██████████| 30000/30000 [00:01<00:00, 20592.26it/s]

Accuracy      : 0.6155
BERTScore F1  : 0.9842
ROUGE-L       : 0.6247
Token F1      : 0.6155
BLEU          : 0.1095


In [6]:
from transformers import BlipProcessor, BlipForQuestionAnswering
from peft import PeftModel, PeftConfig
import torch

# Load processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

# Load base model
base_model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, "/kaggle/input/blip-16-16-no-dense/transformers/default/1").to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): BlipForQuestionAnswering(
      (vision_model): BlipVisionModel(
        (embeddings): BlipVisionEmbeddings(
          (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        )
        (encoder): BlipEncoder(
          (layers): ModuleList(
            (0-11): 12 x BlipEncoderLayer(
              (self_attn): BlipAttention(
                (dropout): Dropout(p=0.0, inplace=False)
                (qkv): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=2304, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_features=2304, bias=False)
                  )
   

In [7]:
from PIL import Image
import pandas as pd
import torch
from PIL import Image
from tqdm import tqdm
from transformers import BlipProcessor, BlipForQuestionAnswering
from peft import PeftModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load your test CSV (replace with actual path if needed)
# test_df = pd.read_csv("/kaggle/input/combined-vqa/flattened_vqa_with_metadata.csv")  # or whatever test set you want
# test_df = test_df.sample(n=30000, random_state=42).copy()  # 🔁 Adjust sample size if needed

import pandas as pd

# Path to your .txt file
txt_path = "/kaggle/input/d/henrychibueze/vqa-dataset/vaq2.0.TestImages.txt"

# Read the file
with open(txt_path, "r") as f:
    lines = f.readlines()

# Parse lines
data = []
for line in lines:
    try:
        image_id, rest = line.strip().split("\t")
        question, answer = rest.rsplit("?", 1)
        question = question.strip() + "?"
        answer = answer.strip()
        image_path = f"/kaggle/input/d/henrychibueze/vqa-dataset/val2014-resised/{image_id.split('#')[0]}"  # update this base path
        data.append({
            "image_path": image_path,
            "question": question,
            "ground_truth_answer": answer
        })
    except Exception as e:
        print(f"Error parsing line: {line} | {e}")

# Create DataFrame
test_df = pd.DataFrame(data)
test_df = test_df.sample(n=1000, random_state=42).copy()  # 🔁 Adjust sample size if needed

# Prepare predictions
predictions = []

for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Running Inference"):
    try:
        image_path = row['image_path']
        question = str(row['question'])

        image = Image.open(image_path).convert("RGB")

        # Preprocess
        inputs = processor(image, question, return_tensors="pt").to(device)

        # Generate answer
        with torch.no_grad():
            output = model.generate(**inputs)

        answer = processor.decode(output[0], skip_special_tokens=True)
        predictions.append(answer)
    except Exception as e:
        print(f"❌ Error with {image_path}: {e}")
        predictions.append("")

# Save predictions to CSV
test_df['predicted_answer'] = predictions
test_df.to_csv("blip-vqa_16-16-no-dense-finetuned_predictions.csv", index=False)
print("✅ Predictions saved to blip_8-16-qkv-finetuned_predictions.csv")


Running Inference: 100%|██████████| 1000/1000 [01:53<00:00,  8.80it/s]

✅ Predictions saved to blip_8-16-qkv-finetuned_predictions.csv


In [3]:
!pip install bert-score rouge-score nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.9 MB/s eta 0:00:00:00:0100:01
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=1dddd3eb2f26efafde4782bfb74b0723724287036991098a1c3111609cb37f90
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1

In [4]:
import pandas as pd
from sklearn.metrics import f1_score
from bert_score import score as bert_score
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
# from bart_score import BARTScorer
from tqdm import tqdm

tqdm.pandas()


In [9]:
# Load CSV
df = pd.read_csv("/kaggle/working/blip-vqa_16-16-no-dense-finetuned_predictions.csv")  # Replace with your CSV file path

# Clean text (optional but recommended)
def normalize_text(s):
    return str(s).strip().lower()

df['ground_truth_answer'] = df['ground_truth_answer'].astype(str).apply(normalize_text)
df['predicted_answer'] = df['predicted_answer'].astype(str).apply(normalize_text)

# Accuracy
accuracy = (df['ground_truth_answer'] == df['predicted_answer']).mean()

# BERTScore (Precision/Recall/F1)
P, R, F1 = bert_score(df['predicted_answer'].tolist(), df['ground_truth_answer'].tolist(), lang='en', verbose=True)
bert_f1 = F1.mean().item()

# ROUGE Score
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
df['rougeL'] = df.progress_apply(lambda row: rouge.score(row['ground_truth_answer'], row['predicted_answer'])['rougeL'].fmeasure, axis=1)
avg_rougeL = df['rougeL'].mean()

# F1 Score (Token-level macro F1)
def compute_token_f1(pred, gt):
    pred_tokens = pred.split()
    gt_tokens = gt.split()
    common = set(pred_tokens) & set(gt_tokens)
    if not pred_tokens or not gt_tokens:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(gt_tokens)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

df['token_f1'] = df.progress_apply(lambda row: compute_token_f1(row['predicted_answer'], row['ground_truth_answer']), axis=1)
avg_token_f1 = df['token_f1'].mean()

# BLEU Score
smooth_fn = SmoothingFunction().method1
df['bleu'] = df.progress_apply(lambda row: sentence_bleu([row['ground_truth_answer'].split()], row['predicted_answer'].split(), smoothing_function=smooth_fn), axis=1)
avg_bleu = df['bleu'].mean()

# # BARTScore (optional; needs large model and GPU)
# bart_scorer = BARTScorer(device='cuda' if torch.cuda.is_available() else 'cpu', checkpoint='facebook/bart-large-cnn')
# df['bartscore'] = df.progress_apply(lambda row: bart_scorer.score([row['answer']], [row['ground_truth']])[0], axis=1)
# avg_bartscore = df['bartscore'].mean()

print(f"Accuracy      : {accuracy:.4f}")
print(f"BERTScore F1  : {bert_f1:.4f}")
print(f"ROUGE-L       : {avg_rougeL:.4f}")
print(f"Token F1      : {avg_token_f1:.4f}")
print(f"BLEU          : {avg_bleu:.4f}")
# print(f"BARTScore     : {avg_bartscore:.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/16 [00:00<?, ?it/s]

done in 0.17 seconds, 5745.83 sentences/sec


100%|██████████| 1000/1000 [00:00<00:00, 19398.59it/s]

Accuracy      : 0.8520
BERTScore F1  : 0.9993
ROUGE-L       : 0.8520
Token F1      : 0.8520
BLEU          : 0.1515


In [5]:
# Load CSV
df = pd.read_csv("/kaggle/working/blip_16-16-no-dense-finetuned_predictions.csv")  # Replace with your CSV file path

# Clean text (optional but recommended)
def normalize_text(s):
    return str(s).strip().lower()

df['answer'] = df['answer'].astype(str).apply(normalize_text)
df['predicted_answer'] = df['predicted_answer'].astype(str).apply(normalize_text)

# Accuracy
accuracy = (df['answer'] == df['predicted_answer']).mean()

# BERTScore (Precision/Recall/F1)
P, R, F1 = bert_score(df['predicted_answer'].tolist(), df['answer'].tolist(), lang='en', verbose=True)
bert_f1 = F1.mean().item()

# ROUGE Score
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
df['rougeL'] = df.progress_apply(lambda row: rouge.score(row['answer'], row['predicted_answer'])['rougeL'].fmeasure, axis=1)
avg_rougeL = df['rougeL'].mean()

# F1 Score (Token-level macro F1)
def compute_token_f1(pred, gt):
    pred_tokens = pred.split()
    gt_tokens = gt.split()
    common = set(pred_tokens) & set(gt_tokens)
    if not pred_tokens or not gt_tokens:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(gt_tokens)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

df['token_f1'] = df.progress_apply(lambda row: compute_token_f1(row['predicted_answer'], row['answer']), axis=1)
avg_token_f1 = df['token_f1'].mean()

# BLEU Score
smooth_fn = SmoothingFunction().method1
df['bleu'] = df.progress_apply(lambda row: sentence_bleu([row['answer'].split()], row['predicted_answer'].split(), smoothing_function=smooth_fn), axis=1)
avg_bleu = df['bleu'].mean()

# # BARTScore (optional; needs large model and GPU)
# bart_scorer = BARTScorer(device='cuda' if torch.cuda.is_available() else 'cpu', checkpoint='facebook/bart-large-cnn')
# df['bartscore'] = df.progress_apply(lambda row: bart_scorer.score([row['answer']], [row['ground_truth']])[0], axis=1)
# avg_bartscore = df['bartscore'].mean()

print(f"Accuracy      : {accuracy:.4f}")
print(f"BERTScore F1  : {bert_f1:.4f}")
print(f"ROUGE-L       : {avg_rougeL:.4f}")
print(f"Token F1      : {avg_token_f1:.4f}")
print(f"BLEU          : {avg_bleu:.4f}")
# print(f"BARTScore     : {avg_bartscore:.4f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

computing greedy matching.


  0%|          | 0/469 [00:00<?, ?it/s]

done in 7.75 seconds, 3869.77 sentences/sec


100%|██████████| 30000/30000 [00:01<00:00, 21152.51it/s]

Accuracy      : 0.7225
BERTScore F1  : 0.9880
ROUGE-L       : 0.7323
Token F1      : 0.7227
BLEU          : 0.1285
